In [98]:
# !pip install urllib
import requests
from bs4 import BeautifulSoup
import re
import os 
import time
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv

In [99]:
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [100]:
print(fileDir)

C:\Users\khushbu\Assignments\Ads_MidTerm


In [101]:
baseUrl='https://freddiemac.embs.com/FLoan/'
postUrl='Data/download.php'

In [102]:
def createCredentialData(user, passwd):
    creds={'username': user,'password': passwd}
    return creds

In [103]:
def getFilesFromFreddieMac(cred,quater,year,quaterTwo,yearTwo):
    c=cred
    ## We are using inside WITH BLock so that session is closed ASAP with BLock is exited 
    with requests.Session() as s:
        ## Step 1 routing to auth.php Site with the proper crentials 
        urlOne = s.post(baseUrl+"secure/auth.php", data=cred) 
        if "Please log in" in urlOne.text:
        ## IF CREDENTIALS are not valid Throw Alert 
            print("Alert: Invalid Credentials, Please try again or sign up on below site \n https://freddiemac.embs.com/FLoan/Bin/loginrequest.php")
        else:
            print("Step1: Logged in")
        ## Sterp 2 Preparing the data for to Accept terms and Conditions 
            pay2={'accept': 'Yes','acceptSubmit':'Continue','action':'acceptTandC'}
            finalUrl=s.post(baseUrl +"Data/download.php",pay2)
            if "Loan-Level Dataset" in finalUrl.text:
                      print("Step2 : Terms and Conditions Accepted")
                      soup = BeautifulSoup(finalUrl.content, "html.parser")   
                      links_list = soup.findAll('a')
                      print("Step3: Filtered the Sample Files with Condition =" + year)
                      print("Status::::::::::")
                      for ele in links_list:
        ## Filtering the ZIp files = 2005 
                         if 'historical' in ele.get_text():
                            if(ele.get_text()[-8:-4] == year):
                                    print(ele.get_text()[-8:-4])
                                    if(ele.get_text()[-10:-8] == quater):
                                        print(ele.get_text()[-10:-8])
                                        
                                        tempUrl = baseUrl+"Data/"+ele.get('href')                         
                                        b =time.time()
                                        downloadUrl=s.post(tempUrl) ## return type = Response
                                        e=time.time()
                                        print(tempUrl + " took "+ str(e-b)+" sec")
                                        with ZipFile(BytesIO(downloadUrl.content)) as zfile:
                                              zfile.extractall(os.path.join(fileDir, 'adsDataRepo/'+'Historical_data_'+ele.get_text()[-8:-4]+'/'))
                                              print("File "+ ele.get_text()+" Downloaded")
                            
                            if(ele.get_text()[-8:-4] == yearTwo):
                                    print(ele.get_text()[-8:-4])
                                    if(ele.get_text()[-10:-8] == quaterTwo):
                                        print(ele.get_text()[-10:-8])
                                        
                                        tempUrl = baseUrl+"Data/"+ele.get('href')                         
                                        b =time.time()
                                        downloadUrl=s.post(tempUrl) ## return type = Response
                                        e=time.time()
                                        print(tempUrl + " took "+ str(e-b)+" sec")
                                        with ZipFile(BytesIO(downloadUrl.content)) as zfile:
                                              zfile.extractall(os.path.join(fileDir, 'adsDataRepo/'+'Historical_data_'+ele.get_text()[-8:-4]+'/'))
                                              print("File "+ ele.get_text()+" Downloaded")
                                
                                              
    
            else:
                print("Alert: Please Check the rerouting action suffix")
        
        ##To scrape the data from the Site finalUrl.       
            

In [104]:
def preProcessData(inputQuater,inputYear,inputQuaterTwo,inputYearTwo):
    print("pre-process data")
    if(os.path.exists(fileDir+'/adsDataRepo/')):
        trainingDataFile = glob.glob(fileDir+'/adsDataRepo/'+'Historical_data_'+inputYear+'/historical_data1_'+inputQuater+inputYear+'.txt')
        testingDataFile = glob.glob(fileDir+'/adsDataRepo/'+'Historical_data_'+inputYearTwo+'/historical_data1_'+inputQuaterTwo+inputYearTwo+'.txt')
        headerNames = ['CreditScore','FirstPaymentDate','FirstTimeHomeBuyerFlag','MaturityDate','MSA','MIP','NumberOfUnits',
                         'OccupancyStatus','OCLTV','DTI','OriginalUPB','OLTV','OriginalInterestRate','Channel','PrepaymentPenaltyFlag',
                         'ProductType','PropertyState','PropertyType','PostalCode','LoanSequenceNumber','LoanPurpose',
                         'OriginalLoanTerm','NumberOfBorrowers','SellerName','ServicerName','SuperConformingFlag']
        with open(trainingDataFile[0]) as f:
            dataf = pd.read_table(f, sep='|', low_memory=False, header=None,lineterminator='\n', names= headerNames)
            cleandataOne = originationDatacleaning(dataf)
            cleandataOne.to_csv("Origination_Clean_"+inputQuater+inputYear+".csv",index=False)
            print("training data cleaned, CSV Created")
       
        with open(testingDataFile[0]) as f:
            dataf = pd.read_table(f, sep='|', low_memory=False, header=None,lineterminator='\n', names= headerNames)
            cleandataTwo = originationDatacleaning(dataf)
            cleandataTwo.to_csv("Origination_Clean_"+inputQuaterTwo+inputYearTwo+".csv",index=False)
            print("testing data cleaned, CSV Created")
        return cleandataOne,cleandataTwo

In [111]:
def originationDatacleaning(dataf):
    dataf['CreditScore'].replace('   ',301,inplace=True)
    dataf['FirstTimeHomeBuyerFlag'].fillna('X',inplace=True) 
    dataf['MSA'].fillna(0, inplace=True) 
    dataf['MIP'].replace('', np.NaN).fillna(0, inplace=True)
    dataf['NumberOfUnits'].fillna(0,inplace=True)
    dataf['OccupancyStatus'].fillna('X',inplace=True)
    dataf['OCLTV'].fillna(0,inplace=True)
    dataf['DTI'].replace('   ',0,inplace=True)
    dataf['OriginalUPB'].replace('', np.NaN).fillna(0,inplace=True)
    dataf['OLTV'].fillna(0,inplace=True)
    dataf['OriginalInterestRate'].fillna(0,inplace=True)
    dataf['Channel'].fillna('X',inplace=True)
    dataf['PrepaymentPenaltyFlag'].fillna('X',inplace=True)
    dataf['ProductType'].fillna('XXXXX',inplace=True)
    dataf['PropertyState'].fillna('XX',inplace=True)
    dataf['PropertyType'].fillna('XX',inplace=True)
    dataf['PostalCode'].replace('', np.NaN).fillna(0,inplace=True)
    dataf['LoanSequenceNumber'].replace('', np.NaN).fillna(0,inplace=True)
    dataf['LoanPurpose'].fillna('X',inplace=True)
    dataf['OriginalLoanTerm'].replace('', np.NaN).fillna(0,inplace=True)
    dataf['NumberOfBorrowers'].fillna('01',inplace=True)
    dataf['SellerName'].fillna('X',inplace=True)
    dataf['ServicerName'].fillna('X',inplace=True)
    dataf['SuperConformingFlag'].fillna('X',inplace=True)
    
    #assingning data 
    
    dataf[['FirstTimeHomeBuyerFlag','OccupancyStatus','Channel','PrepaymentPenaltyFlag','ProductType','PropertyState','PropertyType','LoanSequenceNumber','LoanPurpose',
           'SellerName','ServicerName','SuperConformingFlag']]=dataf[['FirstTimeHomeBuyerFlag','OccupancyStatus','Channel','PrepaymentPenaltyFlag','ProductType','PropertyState','PropertyType','LoanSequenceNumber','LoanPurpose',
           'SellerName','ServicerName','SuperConformingFlag']].astype('str')
    dataf[['OriginalInterestRate']]=dataf[['OriginalInterestRate']].astype('float64')
    dataf[['OriginalUPB']]= dataf[['OriginalUPB']].astype('int64')
    
    #missinganalysis(dataf)
    
    #
    return dataf
    '''As we can see we have the below Null Values presnt in the Data for all the Years (Only varying the Counts )
                       MSA           
    FirstTimeHomeBuyerFlag           
     PrepaymentPenaltyFlag          
         NumberOfBorrowers 
    We can ignore''' 

In [112]:
def main():
    creds=createCredentialData("maiti.t@husky.neu.edu","qRX7JXN4")
    inputQuater= raw_input('Enter Quater')
    inputYear= raw_input('Enter year')
    print(inputQuater[1:2])
    #getFilesFromFreddieMac(creds,inputQuater,inputYear)
    if(int(inputQuater[1:2])<4):
        temp = int(inputQuater[1:2])
        temp += 1
        print(temp)
        t=str(temp)
        newQuater="Q"+t
        inputQuaterTwo=newQuater
        #getFilesFromFreddieMac(creds,inputQuater,inputYear,inputQuaterTwo,inputYear)
        preProcessData(inputQuater,inputYear,inputQuaterTwo,inputYear)
    else:
        year = int(inputYear)+1
        inputYearTwo=str(year)
        inputQuaterTwo="Q1"
        #getFilesFromFreddieMac(creds,inputQuater,inputYear,inputQuaterTwo,inputYearTwo)
        preProcessData(inputQuater,inputYear,inputQuaterTwo,inputYearTwo)
    
if __name__ == '__main__':
    main()   

Enter QuaterQ1
Enter year2007
1
2
pre-process data
training data cleaned, CSV Created
testing data cleaned, CSV Created


## 